In [22]:
# Step 1: Import necessary libraries
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from utils.classification_data import linear_data, checkerboard_data, power_line_data, microgrid_data, make_double_cake_data
from sklearn.model_selection import train_test_split
from pennylane import numpy as np
import pennylane as qml
from pennylane import numpy as np
import matplotlib as mpl
from pennylane import numpy as np
from sklearn.cluster import SpectralClustering, KMeans
from collections import deque
import pandas as pd

#data = checkerboard_data(2)

## Extract features and target
#features = np.asarray(data.drop(columns=['target']))
#target = np.asarray(data['target'])
#target = target % 2
#target = 2 * target - 1

#X, x_test, Y, y_test = train_test_split(
#    features, target, test_size=0.2, random_state=42
#)

In [23]:
data = pd.read_csv('data/balanced_quantum_dataset.csv')
features = np.asarray(data.drop(columns=['label']))
target = np.asarray(data['label'])

X, x_test, Y, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [24]:
import pennylane as qml

circuit_executions = 0
def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])

def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)


def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

dev = qml.device("default.qubit", wires=5, shots=None)
wires = dev.wires.tolist()

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    global circuit_executions
    circuit_executions += 1
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]


In [25]:
params = random_params(num_wires=5, num_layers=6)

In [26]:
kernel_value = kernel(X[0], X[1], params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")

The kernel value between the first and second datapoint is 0.227


In [27]:
classes = np.unique(Y)
n_clusters = 8

centroids = []
class_centroids = []
centroid_labels = []
for c in classes:
    class_data = X[np.where(Y == c)[0]]
    centroids.append(np.mean(class_data, axis=0))
    class_centroids.append([np.mean(cluster, axis=0) for cluster in np.array_split(class_data, n_clusters)])
    centroid_labels.extend([[c] * n_clusters])

In [28]:
def centroid_kernel_matrix(X, centroid, ckernel):
    
    kernel_matrix = []

    for i in range(len(X)):
        kernel_matrix.append(ckernel(centroid, X[i]))

    return np.array(kernel_matrix)

In [29]:

def centroid_target_alignment(X, Y, centroid, kernel, l = 0.1, assume_normalized_kernel=False, rescale_class_labels=True):
   
    Y = np.asarray(Y)
    K = centroid_kernel_matrix(X, centroid, kernel)
    numerator = l * np.sum(Y * K)  
    denominator = np.sqrt(np.sum(K**2) * np.sum(Y**2))

    TA = numerator / denominator

    return TA

In [30]:
def loss(X, Y, centroid, kernel, params, lambda_kao = 0.01):
    TA = centroid_target_alignment(X, Y, centroid, kernel)
    r = lambda_kao * np.sum(params ** 2)
    return 1 - TA + r

In [31]:
class1_centroids = class_centroids[0]
class2_centroids = class_centroids[1]

centroid1_labels = centroid_labels[0]
centroid2_labels = centroid_labels[1]

centroid1 = centroids[0]
centroid2 = centroids[1]

In [32]:
main_centroid = True
kao_class = 1
opt = qml.GradientDescentOptimizer(0.2)
circuit_executions = 0
params = random_params(num_wires=5, num_layers=6)
for i in range(500):
    
    if main_centroid:
        if kao_class == 1:
            cost = lambda _params: loss(
                                
                                        class1_centroids, 
                                        centroid1_labels,
                                        centroid1,
                                        lambda x1, x2: kernel(x1, x2, params),
                                        _params
                                        )
            kao_class = 2
        else:

            cost = lambda _params: loss(
                                
                                        class2_centroids, 
                                        centroid2_labels,
                                        centroid2,
                                        lambda x1, x2: kernel(x1, x2, params),
                                        _params
                                        )
            kao_class = 1
            main_centroid = False

    else:

    #    cost = lambda _params: -qml.kernels.target_alignment(
    #                                                            class1_centroids + class2_centroids,
    #                                                            centroid1_labels,
    #                                                            lambda x1, x2: kernel(x1, x2, _params),
    #                                                            assume_normalized_kernel=True,
    #                                                        )
        kao_class = 1
        main_centroid = True

    #print(params)
    #print(cost(params), main_centroid, kao_class)
    params = opt.step(cost, params)
    
    
    #if (i + 1) % 50 == 0:
print(f"Circuit Executions: {circuit_executions}") 
current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
print(f"Alignment = {current_alignment:.3f}")


Circuit Executions: 4000
Alignment = 0.484


In [33]:
from sklearn.svm import SVC
# First create a kernel with the trained parameter baked into it.
trained_kernel = lambda x1, x2: kernel(x1, x2, params)

# Second create a kernel matrix function using the trained kernel.
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

# Note that SVC expects the kernel argument to be a kernel matrix function.
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y) 

KeyboardInterrupt: 

In [13]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [14]:
accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [15]:
accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [17]:
circuit_executions = 0

params = random_params(num_wires=5, num_layers=6)
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    subset = np.random.choice(list(range(len(X))), 8)
    # Define the cost function for optimization
    cost = lambda _params: -qml.kernels.target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
print(f"Circuit Executions: {circuit_executions}") 
current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
print(f"Alignment = {current_alignment:.3f}")

Circuit Executions: 14000
Alignment = 0.494


In [18]:
from sklearn.svm import SVC
# First create a kernel with the trained parameter baked into it.
trained_kernel = lambda x1, x2: kernel(x1, x2, params)

# Second create a kernel matrix function using the trained kernel.
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

# Note that SVC expects the kernel argument to be a kernel matrix function.
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

In [19]:
accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [20]:
accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [21]:
circuit_executions

42611